In [ ]:
pip install termcolor

In [1]:
import pandas as pd
import sqlite3
from itertools import combinations
from termcolor import colored

<b>Benefit Tier Interpretation - required for analysis</b>

In [2]:
fileName = "Datasets\\Travel_Benefit_Tier_Understanding 11.06.20.xlsx"
#fileName = "Datasets\\Benefit_Tier_Understanding.xlsx"
#fileName = "Datasets\\Benefit_Tier_Understanding_20200929.xlsx"
metadata_df = pd.read_excel(fileName,sheet_name=0, dtype=str, na_filter=False)

metadata_df.head()

,Benefit Tier,Member_Of,Not_Member_Of,Exclude_Benefit_Tiers,Community,Ignore_Community,MP_Ben_Prog_Name,RS_Ben_Prog_Name,RS_Ben_Prog_Name_Not,ONE,Category_Golf,Category_Social,Mbrship_type_desc_contains,Mbrship_type_desc_equals,Mbrship_type_desc_does_not_have
0,CLUBBENEFITS,,"03015,02727",,,,,ClubBenefits,,,,,,,
1,WORLDVANTAGE,,"03015,02727",,,,,WorldVantage,,,,,,,
2,BRONZE TRAVEL,,"03015,02727",,,,Associate Bronze,,"ClubBenefits,WorldVantage,Alumni Travel",,,,,,
3,SILVER TRAVEL,,"03015,02727",,,,Associate Silver,,"ClubBenefits,WorldVantage,Alumni Travel",,,,,,
4,ASSOCIATE ALUMNI,,"03015,02727",,,,Associate Alumni,,"ClubBenefits,WorldVantage,Alumni Travel",,,,,,


In [3]:
# test = metadata_df[['Benefit Tier']]

<b>Benefit Tier Query</b>

In [4]:
#Benefit Tier specific query
query_df = pd.read_csv("output/benefit_tier_query_20201111.csv", dtype=str)
query_df['benefit_tier'] = query_df['benefit_tier'].str.upper()
query_df['benefit_tier'] = query_df['benefit_tier'].str.strip()

<b>Non Resigned Memberships</b>

In [5]:
# Non resigned membership data from Reltio
# df_membership = pd.read_csv("Datasets/dump_membership_reltio_202010271953.csv", dtype=str)
df_membership = pd.read_csv("Datasets/non_resigned_membership_prod_20201111_v2.csv", dtype=str)


In [6]:
query_df.columns

Index(['benefit_tier', 'query'], dtype='object')

In [7]:
%%time
conn = sqlite3.connect("communityOverlap.db")
cur = conn.cursor()

Wall time: 13 ms


In [8]:
benefit_tiers = list(metadata_df[['Benefit Tier']].values.ravel())
benefit_tiers

['CLUBBENEFITS',
 'WORLDVANTAGE',
 'BRONZE TRAVEL',
 'SILVER TRAVEL',
 'ASSOCIATE ALUMNI',
 'ASSOCIATE PLUS',
 'ASSOCIATE GOLD',
 'SIGNATURE GOLD DINING',
 'SIGNATURE GOLD GOLF',
 'SIGNATURE GOLD UNLIMITED',
 'SIGNATURE GOLD SPOUSE GOLF',
 'SIGNATURE GOLD SPOUSE UNLIMITED',
 'PLAY AWAY SELECT',
 'ALUMNI TRAVEL',
 'EWGA']

In [9]:
%%time
overlap_found = []
overlap_not_computed = []
df_membership.to_sql("df", conn, if_exists="replace")        
benefit_tier_combinations = [comb for comb in combinations(benefit_tiers, 2)]
for combination in benefit_tier_combinations:
#         if community == 'ATLANTA':
#             print(combination)   

    benefit_tier0 = combination[0]
    benefit_tier0 = benefit_tier0.replace('  ', ' ')
    benefit_tier1 = combination[1]
    benefit_tier1 = benefit_tier1.replace('  ', ' ')
    query0 = ''
    query1 = ''
    if len(query_df[query_df['benefit_tier'] == benefit_tier0]['query']) > 0:
        query0 = query_df[query_df['benefit_tier'] == benefit_tier0]['query'].iloc[0]
    else:
        continue
    if len(query_df[query_df['benefit_tier'] == benefit_tier1]['query']) > 0:
        query1 = query_df[query_df['benefit_tier'] == benefit_tier1]['query'].iloc[0]
    else:
        continue 

    rs0 = pd.read_sql_query(query0, conn)
    rs1 = pd.read_sql_query(query1, conn)

#         if ((benefit_tier0 == 'ONE GOLF - PREMIER - ATLANTA' or benefit_tier0 == 'ONE GOLF - NONPREMIER - ATLANTA') & (benefit_tier1 == 'ONE GOLF - PREMIER - ATLANTA' or benefit_tier1 == 'ONE GOLF - NONPREMIER - ATLANTA')):
#             print(query0)


    if len(rs0.index) == 0 or len(rs1.index) == 0:
        #print("Overlap not computed for," + community + "," + benefit_tier0 + ", " + benefit_tier1)
        overlap_not_computed.append({
            'benefit_tier0': benefit_tier0,
            'benefit_tier1': benefit_tier1
        })
    else:
        intersection = list(set(rs0['ID']) & set(rs1['ID']))
        if len(intersection) > 0:
            print(colored(benefit_tier0 + "," + benefit_tier1,'red'))
            overlap_found.append({
                'benefit_tier0': benefit_tier0,
                'benefit_tier1': benefit_tier1                    
            })

if len(overlap_found) == 0:
    print("No overlaps detected")
else:
    print("Overlap count: " + str(len(overlap_found)))

No overlaps detected
Wall time: 7min 41s


In [ ]:
if len(overlap_found) > 0:
    overlap_found_df = pd.DataFrame(overlap_found, columns=overlap_found[0].keys())
    overlap_found_df.to_csv("output/community_benefits_overlaps.csv", index=False)
else:
    print("No overlaps found, output file not generated")